Before starting we need to setup some configuration, like which AI backend to use.

When using the kernel for AI requests, the kernel needs some settings like URL and
credentials to the AI models. We will need access to one or several Oobabooga models, and to an OpenAI model for multiconnector advanced chat tests.

The following code will ask a few questions and save the settings to a local
[config/settings.json](config/settings.json) configuration file, under the [config](config) folder. 
Alternatively, rename and edit the [config/settings.example.json](config/settings.example.json) file.
**Please keep the file safe.**

### Step 1 Import configuration packages and classes

In order to build the settings file, we need a couple helpers.

In [3]:
//Install Nuget Packages
#r "nuget: Microsoft.Extensions.Configuration"
#r "nuget: Microsoft.Extensions.Configuration.Json"
#r "nuget: Microsoft.Extensions.Configuration.Binder"

//Load support classes
#!import config/Settings.cs
#r "nuget: MyIA.SemanticKernel.Connectors.AI.MultiConnector"

// Create a dictionary to hold all configurations
var allConfigs = new Dictionary<string, object>();

Installed Packages Microsoft.Extensions.Configuration, 8.0.0-rc.1.23419.4 Microsoft.Extensions.Configuration.Binder, 8.0.0-rc.1.23419.4 Microsoft.Extensions.Configuration.Json, 8.0.0-rc.1.23419.4 MyIA.SemanticKernel.Connectors.AI.MultiConnector, 0.33.6

### Step 2 Define your OpenAI configuration

Enter your OpenAI credentials.
ChatGPT will be used to chat with Oobabooga in advanced Oobabooga chat capabilities tests, and as a primary connector for the multiconnector tests.
If you only plan on testing Oobabooga completion on its own, you can skip this step.

Run the following code. If you need to find the value and copy and paste, you can
re-run the code and continue from where you left off.

In [2]:
using MyIA.SemanticKernel.Connectors.AI.MultiConnector.Configuration;

// Creating the Open AI configuration object
var openAIConfig = new OpenAIConfiguration();
allConfigs.Add("OpenAI", openAIConfig);

// Asking for the Open AI API key

var openAIKey = await InteractiveKernel.GetInputAsync("Please enter your OpenAI API key");
openAIConfig.ApiKey = openAIKey;

openAIConfig.ModelId = "text-davinci-003";
openAIConfig.ChatModelId = "gpt-3.5-turbo";


### Step 3 Define your primary Oobabooga configuration

To test Ooobabooga completion providers, define your Oobabooga endpoint settings. You can skip this part if you want to skip testing the oobabooga connector indivudally and you want to use oobabooga as part of a multiconnector where ChatGPT serves the role of a primary connector.

Run the following code. If you need to find the value and copy and paste, you can
re-run the code and continue from where you left off.

In [ ]:
using MyIA.SemanticKernel.Connectors.AI.MultiConnector.Configuration;

// Creating the Oobabooga AI configuration object
var oobaboogaConfig = new OobaboogaConnectorConfiguration();
allConfigs.Add("Oobabooga", oobaboogaConfig);

var oobaboogaEndpoint = await InteractiveKernel.GetInputAsync("Please enter your Oobabooga endpoint");
oobaboogaConfig.EndPoint = oobaboogaEndpoint;

var oobaboogaBlockingPort = await InteractiveKernel.GetInputAsync("Please enter your Oobabooga API Blocking port (default 5000)");
oobaboogaConfig.BlockingPort = int.Parse(oobaboogaBlockingPort);

var oobaboogaStreamingPort = await InteractiveKernel.GetInputAsync("Please enter your Oobabooga API Streaming port (default 5005)");
oobaboogaConfig.StreamingPort = int.Parse(oobaboogaStreamingPort);



### Step 4 Define your Multiconnector secondary Oobabooga connectors

To test the multiconnector, your OpenAI model will be used as a primary connector.
Now you need to define the settings for all your Oobabooga secondary connectors: they will be similar to your primary Oobabooga connector, but you must give them names (typically those of the corresponding models).

Run the following code. If you need to find the value and copy and paste, you can
re-run the code and continue from where you left off.

In [ ]:
using MyIA.SemanticKernel.Connectors.AI.MultiConnector.Configuration;

// Creating a MultiConnector configuration object
var multiOobaboogaConfig = new MultiOobaboogaConnectorConfiguration();
allConfigs.Add("MultiConnector", multiOobaboogaConfig);

var defaultEndpoint = await InteractiveKernel.GetInputAsync("Enter the default endpoint for your Oobabooga connectors");
multiOobaboogaConfig.OobaboogaEndPoint = defaultEndpoint;
string addMore = "yes";
while (addMore.ToLower() == "yes")
{
    var connectorName = await InteractiveKernel.GetInputAsync("Please enter the name of this secondary Oobabooga connector");
    var blockingPort = await InteractiveKernel.GetInputAsync("Please enter the Blocking Port for this connector (default 5000)");
    var streamingPort = await InteractiveKernel.GetInputAsync("Please enter the Streaming Port for this connector (default 5005)");
    var endpoint = await InteractiveKernel.GetInputAsync("If different from the default, please enter the endpoint for this connector, otherwise press space and enter");
    var strIsChat = await InteractiveKernel.GetInputAsync("One last question, was this model trained for chat specifically? (yes/no)");
    var isChat = strIsChat.ToLower() == "yes"; 
    var newConnector = new OobaboogaConnectorConfiguration
    {
        Name = connectorName,
        BlockingPort = int.Parse(blockingPort),
        StreamingPort = int.Parse(streamingPort),
        UseChatCompletion = isChat
    };
    if (!string.IsNullOrEmpty(endpoint.Trim()))
    {
        newConnector.EndPoint = endpoint;
    }
    
    multiOobaboogaConfig.OobaboogaCompletions.Add(newConnector);
    multiOobaboogaConfig.IncludedConnectors.Add(connectorName);

    addMore = await InteractiveKernel.GetInputAsync("Would you like to add another connector? (yes/no)");
}


### Step 4: Save Configuration to `settings.json`

This should be enough to get started. Your settings will be saved to [config/settings.json](config/settings.json). You may want to add more parameters depending on your model. Look at the setting.json file in the multiconnector integration test for an example on how to refine your settings.

In [ ]:
using System.Text.Json;
using System.IO;

// Serialize the dictionary to JSON
string jsonString = JsonSerializer.Serialize(allConfigs, new JsonSerializerOptions { WriteIndented = true });

// Save the JSON string to settings.json
File.WriteAllText("config/settings.json", jsonString);

Console.WriteLine("Settings saved to config/settings.json:");
Console.WriteLine(jsonString);

If the code above doesn't show any error, you're good to go and run the other notebooks.

### Resetting the configuration

If you want to reset the configuration and start again, please uncomment and run the code below.
You can also edit the [config/settings.json](config/settings.json) manually if you prefer.

In [ ]:
#!import config/Settings.cs

// Uncomment this line to reset your settings and delete the file from disk.
// Settings.Reset();

Now that your environment is all set up, let's dive into
[how to do basic loading of the Semantic Kernel](01-basic-loading-the-kernel.ipynb).